# MODIS Raster Analysis
Analysis of MODIS snow albedo data with temporal pixel tracking

In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd

## Database Overview

In [ ]:
# Query database for available datasets
# Use MCP PostgreSQL tools to run this query:
# SELECT dataset_name, COUNT(*) as count FROM raster_data GROUP BY dataset_name ORDER BY count DESC;

## Real Raster Visualization

In [ ]:
# Load real MODIS raster file
raster_path = "modis_melt_season_2017_2024/clipped/2017-06-03_MOD10A1.A2017154.h10v03.061.2021273084644_Snow_Albedo_Daily_Tile.tif"

with rasterio.open(raster_path) as src:
    data = src.read(1)
    
plt.figure(figsize=(12, 8))
plt.imshow(data, cmap='viridis')
plt.colorbar(label='Albedo')
plt.title('MODIS Snow Albedo - 2017-06-03')
plt.show()

## Daily Albedo Change (Melt Season 2017-2018)

In [ ]:
# Load multiple rasters for temporal analysis
clipped_dir = Path("modis_melt_season_2017_2024/clipped")
raster_files = list(clipped_dir.glob("2017*Snow_Albedo*.tif"))[:20]  # First 20 files

dates = []
mean_albedo = []

for file in sorted(raster_files):
    with rasterio.open(file) as src:
        data = src.read(1)
        valid_data = data[(data > 0) & (data < 100)]  # Filter valid albedo values
        dates.append(file.stem.split('_')[0])
        mean_albedo.append(np.mean(valid_data))

# Convert to day of year for melt season visualization
day_numbers = list(range(1, len(dates) + 1))

plt.figure(figsize=(10, 6))
plt.plot(day_numbers, mean_albedo, 'b-o', markersize=4)
plt.xlabel('Day of Melt Season')
plt.ylabel('Mean Albedo (%)')
plt.title('Daily Albedo Change - Melt Season 2017')
plt.grid(True, alpha=0.3)
plt.show()

## Database Raster Query

In [ ]:
# Use MCP PostgreSQL to extract raster data:
# SELECT (ST_PixelAsPoints(rast, 1)).*
# FROM raster_data 
# WHERE dataset_name LIKE '%Snow_Albedo%'
# AND date_acquired = '2017-06-03'
# LIMIT 1000;

## Temporal Pixel Analysis

In [ ]:
# Query same pixel across time using MCP PostgreSQL:
# SELECT 
#     date_acquired,
#     ST_Value(rast, 1, 50, 30) as albedo_value
# FROM raster_data 
# WHERE dataset_name LIKE '%Snow_Albedo%'
# AND date_acquired BETWEEN '2017-06-01' AND '2017-08-31'
# ORDER BY date_acquired;

# Example visualization (replace with actual query results)
example_dates = pd.date_range('2017-06-01', '2017-08-31', freq='5D')
example_values = np.random.normal(50, 10, len(example_dates))

plt.figure(figsize=(12, 6))
plt.plot(example_dates, example_values, 'r-o', markersize=3)
plt.xlabel('Date')
plt.ylabel('Pixel Albedo Value')
plt.title('Temporal Change of Single Pixel (Row 50, Col 30)')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()